## Optimization

This task loads the model input dataframes from temporary files written by tasks 1a, 1b & 1c, formulates and solves the optimization model using Gurobi, and writes the results to a temporary file.

In [ ]:
# If running in databricks, fetch Gurobi cloud license key from secrets.
# If unavailable, use Gurobi's free size-limited license to run the job.

try:
    gurobi_params = {
        "CloudAccessID": dbutils.secrets.get("grbcloud", "CloudAccessID"),
        "CloudSecretKey": dbutils.secrets.get("grbcloud", "CloudSecretKey"),
        "LicenseID": int(dbutils.secrets.get("grbcloud", "LicenseID")),
        "CSAppName": dbutils.secrets.get("grbcloud", "CSAppName"),
        "CloudPool": dbutils.secrets.get("grbcloud", "CloudPool"),
    }
except Exception as e:
    print("Databricks secrets are not set up.")
    print(f"Error message: {e}")
    print("Using Gurobi size-limited license.")
    gurobi_params = {}

In [ ]:
import pathlib

model_data_path = pathlib.Path("./model_data/").resolve()

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import gurobipy_pandas as gppd

In [ ]:
feasible_assignments = pd.read_feather(model_data_path / "feasible_assignments.feather")
staff_required = pd.read_feather(model_data_path / "staff_required.feather")
shift_conflicts = pd.read_feather(model_data_path / "shift_conflicts.feather")

In [ ]:
with gp.Env(params=gurobi_params) as env, gp.Model(env=env) as model:

    model.ModelSense = GRB.MAXIMIZE
    assign = gppd.add_vars(
        model,
        feasible_assignments.set_index(["staff_id", "shift_id"]),
        obj=1.0,
        vtype=GRB.BINARY,
        name="assign",
    )
    
    constr_requires = gppd.add_constrs(
        model,
        assign.groupby("shift_id").sum(),
        GRB.LESS_EQUAL,
        staff_required["staff_count"],
        name="staffing",
    )

    df_conflict_vars = (
        shift_conflicts
        .join(assign.rename("assign1"), on=["staff_id", "shift1_id"])
        .join(assign.rename("assign2"), on=["staff_id", "shift2_id"])
        .dropna()
    )

    constr_conflicts = df_conflict_vars.gppd.add_constrs(
        model,
        "assign1 + assign2 <= 1",
        name="conflict",
    )

    model.optimize()

    assignments = assign.gppd.X
    assignments = assignments[assignments == 1.0].reset_index().drop(columns=["assign"])

assignments.to_feather(model_data_path / "assignments.feather")

In [ ]:
assignments.sample(5)